## Personal Tutor Agent

### Tech Stack
- Langchain
- Ollama (model_name)

### Agents Details

In [22]:
# ! pip install -r requirements.txt

In [23]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import BaseTool
from typing import List, Dict, Any

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}
Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

supervisor_model = OllamaLLM(model="deepseek-r1:8b")
tutor_model = OllamaLLM(model="llama3.2:3b")

test_chain = prompt | supervisor_model
test_chain.invoke({"question": "What is LangChain?"})

"<think>\nOkay, so I need to figure out what LangChain is. From the previous explanation, it seems like LangChain is related to language models and chaining them together in a particular way to improve performance.\n\nI remember that sometimes when you have multiple models, you can chain their outputs by using one model's output as input for another. This might be useful because each model could capture different aspects or provide more detailed information.\n\nThe user mentioned that LangChain enhances the performance of models, maybe allowing them to handle longer texts or process more complex tasks. But I'm not entirely sure how exactly this chaining works. Let me think about some examples.\n\nSuppose we have two language models: Model A and Model B. If someone asks a question, Model A might generate an initial response, but it could be too short or miss some important points. Then, the user could ask Model B to expand on that response using the output from Model A as context. This 

In [ ]:
# Create a memory buffer for the tutor agent
tutor_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

tutor_feedback = ConversationBufferMemory(
    memory_key="feedback",
    return_messages=True
)

# Define the tutor agent's prompt
tutor_prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template="""You are an English language tutor. Your role is to:
    1. Provide clear explanations of the lesson materials
    2. Provide constructive feedback to the student after each response
    3. Maintain an encouraging and supportive tone
    4. Keep a list of the student's strengths and weaknesses and add this to the feedback memory

    Lession Materials:
    1. Grammar Basics
    1.1 Identify the subject and predicate in the following sentence: 'The cat sat on the mat.'


    Previous conversation:
    {chat_history}

    Student's input: {input}
    Tutor's response:"""
)

# Create the tutor chain
tutor_chain = LLMChain(
    llm=tutor_model,
    prompt=tutor_prompt,
    memory=tutor_memory,
    verbose=True
)

tutor_chain = prompt | tutor_model

# Define the tutor tool
class TutorTool(BaseTool):
    name: str = "english_tutor"
    description: str = "Use this tool to interact with the English tutor for teaching and feedback"
    
    def _run(self, query: str) -> str:
        return tutor_chain.run(input=query)
    
    def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not implemented")

#Define the tutor tools

class IntroduceLessonTool(BaseTool):
    name: str = "introduce_lesson"
    description: str = "Introduce the lesson material"
    
    def _run(self, lesson_introduction: str) -> str:
        introduction = f"Today's lesson is about: {lesson_introduction}"
        print(introduction)
        return introduction
    
    def _arun(self, lesson_introduction: str) -> str:
        raise NotImplementedError("Async not implemented")

class ReadExerciseTool(BaseTool):
    name: str = "read_exercise"
    description: str = "Read out the first exercise and await a response"
    
    def _run(self, exercise: str) -> str:
        print(f"Exercise: {exercise}")
        return exercise
    
    def _arun(self, exercise: str) -> str:
        raise NotImplementedError("Async not implemented")

class GiveFeedbackTool(BaseTool):
    name: str = "give_feedback"
    description: str = "Give feedback on the response and proceed to the next exercise"
    
    def _run(self, student_response: str) -> str:
        feedback = tutor_chain.run(input=student_response)
        print(f"Feedback: {feedback}")
        return feedback
    
    def _arun(self, student_response: str) -> str:
        raise NotImplementedError("Async not implemented")

class UpdateFeedbackMemoryTool(BaseTool):
    name: str = "update_feedback_memory"
    description: str = "Update the tutor_feedback memory buffer"
    
    def _run(self, feedback: str) -> str:
        tutor_feedback.add_message(feedback)
        print("Feedback memory updated.")
        return "Feedback memory updated."
    
    def _arun(self, feedback: str) -> str:
        raise NotImplementedError("Async not implemented")

# Create tools list for supervisor agent
tools = [
    TutorTool(),
    IntroduceLessonTool(),
    ReadExerciseTool(),
    GiveFeedbackTool(),
    UpdateFeedbackMemoryTool(),
]

# Example usage
# deploy_tool = DeployTutorTool()
# deploy_tool._run("")

# introduce_tool = IntroduceLessonTool()
# introduce_tool._run("Grammar Basics")

# read_tool = ReadExerciseTool()
# read_tool._run("Identify the subject and predicate in the following sentence: 'The cat sat on the mat.'")

# student_response = "The subject is 'The cat' and the predicate is 'sat on the mat.'"
# feedback_tool = GiveFeedbackTool()
# feedback = feedback_tool._run(student_response)

# update_tool = UpdateFeedbackMemoryTool()
# update_tool._run(feedback)


In [ ]:

# Create supervisor agent memory
supervisor_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Initialize the supervisor agent
supervisor_agent = initialize_agent(
    tools,
    supervisor_model,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=supervisor_memory,
    verbose=True
)

# Define the supervisor's system prompt
supervisor_system_prompt = """You are a supervisor agent responsible for managing an English tutoring session. Your role is to:
1. Assess the student's needs and current English level based on tutor_feedback memory.
2. Deploy the English tutor agent to teach the lesson materials.
3. Provide high-level guidance and structure to the session.
4. Once the lesson is complete, review the feedback memory and plan the next lesson accordingly.

Use the english_tutor tool to delegate actual teaching tasks and student interaction.
Always maintain a clear structure and learning objectives for the session."""

### Run the session

In [27]:
# Function to run a tutoring session
def run_tutoring_session(user_input: str) -> str:
    response = supervisor_agent.run(
        input=f"{supervisor_system_prompt}\n\nStudent input: {user_input}"
    )
    return response

# Example usage
if __name__ == "__main__":
    # Example interaction
    student_input = "Hi, I'm an intermediate English learner and I need help with phrasal verbs."
    response = run_tutoring_session(student_input)
    print(response)



> Entering new AgentExecutor chain...
<think>
Okay, so I'm trying to figure out how to handle this tutoring session. The user is an intermediate English learner who wants help with phrasal verbs. My role is as a supervisor agent, so I need to manage the session effectively.

First, I should assess the student's needs based on any feedback from previous lessons stored in the tutor_feedback memory. Wait, but the memory seems empty here. That might mean I don't have specific information about their progress. Hmm, that could be a problem because without knowing their strengths or weaknesses, it's harder to plan the session.

But since there's no feedback available, maybe I should proceed with general guidance. The student specifically asked for help with phrasal verbs, so the lesson material should focus on those. I remember that phrasal verbs are tricky because they change meaning significantly depending on their form. So, the tutor needs to explain how these verbs work and provide exam

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `

**Step-by-Step Explanation and Feedback:**

1. **Understanding the Sentence Structure:**
   - The sentence "The cat sat on the mat" consists of a subject ('the cat') and a predicate ('sat on the mat').

2. **Breaking Down the Predicate:**
   - The predicate includes the verb 'sat' and its accompanying preposition phrase 'on the mat.'

3. **Identifying Parts of Speech:**
   - Recognizing parts of speech such as verbs, adjectives, nouns, and prepositions helps in deconstructing sentence structures.

4. **Next Steps for Improvement:**
   - Practice identifying different types of sentences to understand how subjects and predicates vary.
   - Pay attention to verb forms and the roles of prepositions in altering sentence meaning.

5. **Feedback Summary:**
   - You've demonst